# Step 4: Create Isochrones

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point
import partridge as ptg

In [3]:
import os, sys

In [4]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

##  Load data

In [5]:
path = r'input_folder/cap_haitien_gtfs.zip'

In [6]:
# original graph
original_G = nx.read_gpickle(r"temp\cap_haitien_walk_w_ferries_via_osmnx.pickle")
# original graph with time and snapped origin points
original_G_adv_snap = nx.read_gpickle(r"temp\cap_haitien_walk_w_ferries_via_osmnx_w_time_adv_snap.pickle")

### Load the stops for service0001
service_0001 is on the weekends. 6-29-2019 lands on a weekend.

In [7]:
# from: http://simplistic.me/playing-with-gtfs.html
import datetime

service_ids_by_date = ptg.read_service_ids_by_date(path)
service_ids = service_ids_by_date[datetime.date(2019, 6, 29)]

print(f"service_ids is {service_ids}")

# view lets you filter before you load the feed. For example, below you are filtering by the service_ids
feed = ptg.load_feed(path, view={
    'trips.txt': {
        'service_id': service_ids,
    },
})

service_ids is frozenset({'service_0001'})


In [8]:
feed.stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,stop_0001,NaN,Bonnay Dugal 1,NaN,19.704813,-72.180840,NaN,NaN,0,NaN,NaN,NaN
1,stop_0002,NaN,Cité du Peuple,NaN,19.744087,-72.212227,NaN,NaN,0,NaN,NaN,NaN
2,stop_0003,NaN,Cité Lescot,NaN,19.752174,-72.206665,NaN,NaN,0,NaN,NaN,NaN
3,stop_0004,NaN,Dubreuil,NaN,19.613947,-72.203201,NaN,NaN,0,NaN,NaN,NaN
4,stop_0005,NaN,Pipo-Ville,NaN,19.722265,-72.241341,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
256,stop_0260,NaN,Morne Rouge,NaN,19.708532,-72.273491,NaN,NaN,0,hub_0003,NaN,NaN
257,stop_0261,NaN,Haut Du Cap 1,NaN,19.721527,-72.231148,NaN,NaN,0,NaN,NaN,NaN
258,stop_0262,NaN,Limonade,NaN,19.672407,-72.124825,NaN,NaN,0,hub_0007,NaN,NaN
259,stop_0263,NaN,Carrefour Bas Limbe,NaN,19.703863,-72.379448,NaN,NaN,0,NaN,NaN,NaN


In [9]:
stops = feed.stops[['stop_id','stop_lat','stop_lon']]

In [10]:
stops_gdf_service0001 = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat))

In [11]:
stops_gdf_service0001[:3]

,stop_id,stop_lat,stop_lon,geometry
0,stop_0001,19.704813,-72.180840,POINT (-72.18084 19.70481)
1,stop_0002,19.744087,-72.212227,POINT (-72.21223 19.74409)
2,stop_0003,19.752174,-72.206665,POINT (-72.20667 19.75217)


### Load the stops for service0002
service_0001 is on the weekday. 7-01-2019 lands on a weekday.

In [12]:
# from: http://simplistic.me/playing-with-gtfs.html
import datetime

service_ids_by_date = ptg.read_service_ids_by_date(path)
service_ids = service_ids_by_date[datetime.date(2019, 7, 1)]

print(f"service_ids is {service_ids}")

# view lets you filter before you load the feed. For example, below you are filtering by the service_ids
feed = ptg.load_feed(path, view={
    'trips.txt': {
        'service_id': service_ids,
    },
})

service_ids is frozenset({'service_0002'})


In [13]:
stops = feed.stops[['stop_id','stop_lat','stop_lon']]

In [14]:
stops_gdf_service0002 = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat))

In [15]:
stops_gdf_service0002[:3]

,stop_id,stop_lat,stop_lon,geometry
0,stop_0001,19.704813,-72.180840,POINT (-72.18084 19.70481)
1,stop_0002,19.744087,-72.212227,POINT (-72.21223 19.74409)
2,stop_0003,19.752174,-72.206665,POINT (-72.20667 19.75217)


### Load the health facilities

In [16]:
health_facilities = gpd.read_file(r"input_folder\cap_haitien_health_pts.shp")

In [17]:
health_facilities[:3]

,Code2018,Facility n,Department,Vilcom,Commune,Facility t,MGA,Geolocatio,Geolocat_1,geometry
0,476,HôPital Universitaire Justinien,Nord,11,Cap-Haitien,hopital universitaire,publique,19.762251,-72.206200,POINT (-72.20620 19.76225)
1,573,Centre Sante Communautaire De Ferrier (Csc),Nord'Est,12,Ferrier,dispensaire/centre communautaire de sante,publique,19.616028,-71.779961,POINT (-71.77996 19.61603)
2,593,Centre De Santé De Sainte Suzanne,Nord'Est,32,Sainte-Suzanne,centre de sante sans lit,publique,19.584867,-72.089272,POINT (-72.08927 19.58487)


## Process Isochrones
Only use the two service graphs for creating the isochrones for the health facilities. For the stops, you will read in only the walk graph.

### Health Facilities
We will use the graph with the GTFS lines to calculate the isochrones, but we will choose to ignore drawing the isochrones around the GTFS edges

In [18]:
# read back your graphs from step 2 from you saved pickle
G_service0001 = nx.read_gpickle(r"temp\gtfs_export_cap_haitien_merged_impute_walk_adv_snap_service0001.pickle")
G_service0002 = nx.read_gpickle(r"temp\gtfs_export_cap_haitien_merged_impute_walk_adv_snap_service0002.pickle")

graphs = {'G_service0001': G_service0001, 'G_service0002': G_service0002}
#graphs = {'G_service0002': G_service0002}

In [19]:
%%time
for G in graphs.items():
    GTFS_graph_snapped = gn.pandana_snap(original_G, health_facilities)
    NN_to_graph = list(set(list(GTFS_graph_snapped.NN)))
    # simulate up to a 60-min isochrone
    iso_gdf = gn.make_iso_polys(G[1], NN_to_graph, [3600], edge_buff=700, node_buff=700, weight = 'length', measure_crs = 'epsg:32619', edge_filters = {'mode':'transit'})
    dissolved = iso_gdf.dissolve(by="thresh")
    gdf_out = dissolved.explode()
    gdf_out2 = gdf_out.reset_index()
    # save file
    gdf_out2.to_file(fr"output_folder\isochrones_health_facilities_{G[0]}_60m_adv_snap.shp")

merge all edges and nodes
unary_union
merge all edges and nodes
unary_union
Wall time: 23min 4s


### GTFS stops
We want to represent walkability to the GTFS stops, so we will use the original graph without the GTFS lines when calculating the isochrones

In [20]:
stops_dict = {'stops_gdf_service0001': stops_gdf_service0001,'stops_gdf_service0002': stops_gdf_service0002}

In [21]:
%%time
for stops in stops_dict.items():
    GTFS_graph_snapped = gn.pandana_snap(original_G, stops[1])
    NN_to_stops = list(set(list(GTFS_graph_snapped.NN)))
    
    # simulate up to a 15-min and 30-min isochrone
    # at 3.5 Kmph, 15 min equals 875m
    iso_gdf = gn.make_iso_polys(original_G_adv_snap, NN_to_stops, [900], edge_buff=700, node_buff=700, weight = 'length', measure_crs = 'epsg:32619')
    dissolved = iso_gdf.dissolve(by="thresh")
    gdf_out = dissolved.explode()
    gdf_out2 = gdf_out.reset_index()
    # save file
    gdf_out2.to_file(fr"output_folder\isochrones_{stops[0]}_original_G_15m_adv_snap.shp")
    
    # simulate up to 30-min isochrone
    # at 3.5 Kmph, 30 min equals 1750m
    iso_gdf = gn.make_iso_polys(original_G_adv_snap, NN_to_stops, [1800], edge_buff=700, node_buff=700, weight = 'length', measure_crs = 'epsg:32619')
    dissolved = iso_gdf.dissolve(by="thresh")
    gdf_out = dissolved.explode()
    gdf_out2 = gdf_out.reset_index()
    # save file
    gdf_out2.to_file(fr"output_folder\isochrones_{stops[0]}_original_G_30m_adv_snap.shp")

merge all edges and nodes
unary_union
merge all edges and nodes
unary_union
merge all edges and nodes
unary_union
merge all edges and nodes
unary_union
Wall time: 8min 59s


## Create tables for population count and percentage of population within isochrones

In [22]:
# read in WorldPop and count total polygons
# read in shapefiles that begin with isochrones_

In [23]:
# load origins
origins = gpd.read_file(r"input_folder\cap_haitien_worldpop_pts2.shp")
origins

,VALUE,geometry
0,3.630365,POINT (-72.38833 19.81667)
1,3.881798,POINT (-72.38667 19.81667)
2,4.394933,POINT (-72.38250 19.81667)
3,4.479361,POINT (-72.38167 19.81667)
4,3.633273,POINT (-72.38833 19.81583)
...,...,...
182971,1.172181,POINT (-72.18667 19.49583)
182972,1.175843,POINT (-72.18583 19.49583)
182973,1.067793,POINT (-72.18500 19.49583)
182974,1.201748,POINT (-72.18417 19.49583)


In [24]:
# sum of Worldpop points
total_pop = origins.VALUE.sum()

### for stops

In [25]:
for stops in stops_dict.items():
    d = {'label':[],'population':[]}
    # for 15-min
    isochrone_gdf = gpd.read_file(fr"output_folder\isochrones_{stops[0]}_original_G_15m_adv_snap.shp")
    #isochrone_gdf
    
    points_in_polygons = gpd.overlay(origins, isochrone_gdf, how = 'intersection')
    iso_pop = points_in_polygons.VALUE.sum()
    percent_in_iso = iso_pop/total_pop
    #percent_in_iso
    
    d['label'].append('Total_pop')
    d['population'].append(total_pop)
    
    d['label'].append('pop in 15-min isochrones')
    d['population'].append(iso_pop)
    
    d['label'].append('percentage of pop in 15-min isochrones')
    d['population'].append(percent_in_iso)
    
    # for 30-min
    isochrone_gdf = gpd.read_file(fr"output_folder\isochrones_{stops[0]}_original_G_30m_adv_snap.shp")
    #isochrone_gdf
    
    points_in_polygons = gpd.overlay(origins, isochrone_gdf, how = 'intersection')
    iso_pop = points_in_polygons.VALUE.sum()
    percent_in_iso = iso_pop/total_pop
    #percent_in_iso
    
    d['label'].append('Total_pop')
    d['population'].append(total_pop)
    
    d['label'].append('pop in 30-min isochrones')
    d['population'].append(iso_pop)
    
    d['label'].append('percentage of pop in 30-min isochrones')
    d['population'].append(percent_in_iso)

    d_df = pd.DataFrame(d)

    # save a CSV of Table
    d_df.to_csv(fr"output_folder\table_isochrones_{stops[0]}_stops_adv_snap.csv")

### for health facilities

In [26]:
for G in graphs.items():
   
    d = {'label':[],'population':[]}

    isochrone_gdf = gpd.read_file(fr"output_folder\isochrones_health_facilities_{G[0]}_60m_adv_snap.shp")
    #isochrone_gdf
    
    points_in_polygons = gpd.overlay(origins, isochrone_gdf, how = 'intersection')
    iso_pop = points_in_polygons.VALUE.sum()
    percent_in_iso = iso_pop/total_pop
    #percent_in_iso
    
    d['label'].append('Total_pop')
    d['population'].append(total_pop)
    
    d['label'].append('pop in 60-min isochrones')
    d['population'].append(iso_pop)
    
    d['label'].append('percentage of pop in 60-min isochrones')
    d['population'].append(percent_in_iso)
    
    d_df = pd.DataFrame(d)

    # save a CSV of Table
    d_df.to_csv(fr"output_folder\table_isochrones_{G[0]}_health_adv_snap.csv")